In [37]:
import pandas as pd

### Quick look at the data

I am filtering out rows where the occupation code hasn't been recorded. Does this affect our results?

In [58]:
non_compliance_data = pd.read_csv('./data/afsa/non-compliance-in-personal-insolvencies.csv')
display(non_compliance_data.shape)
#non_compliance_data = non_compliance_data[non_compliance_data['Debtor Occupation Code (ANZSCO)'].notnull()]
display(non_compliance_data.shape)
non_compliance_data.isnull().sum()

/home/nic/code/govhack2018-EZ0/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (17,18,19,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(356500, 23)

(356500, 23)

Unique ID                                0
Calendar Year of Insolvency              0
SA3 of Debtor                            0
SA3 Code of Debtor                       0
GCCSA of Debtor                          0
GCCSA Code of Debtor                     0
State of Debtor                          0
Sex of Debtor                            0
Family Situation                         0
Debtor Occupation Code (ANZSCO)      15886
Debtor Occupation Name (ANZSCO)      15886
Main Cause of Insolvency                 0
Business Related Insolvency              0
Debtor Income                            0
Primary Income Source                    0
Unsecured Debts                          0
Value of Assets                          0
Type of Party                       341629
Non-Compliance Type                 341608
Result of Non-Compliance            342306
Number of Instances                 341608
Outcome of Non-Compliance           345672
Non-Compliance Conviction Result    352942
dtype: int6

### 1. Find all unique family situations in non-compliance-in-personal-insolvencies.csv

In [59]:
display(non_compliance_data['Family Situation'].unique())
family_situations = ['Single with Dependants', 'Couple without Dependants',
       'Couple with Dependants', 'Single without Dependants']

array(['Single with Dependants', 'Couple without Dependants',
       'Couple with Dependants', 'Single without Dependants', 'Unknown',
       'Not Stated'], dtype=object)

### 2. Express each such family situation in terms of the columns in B25 / G25

In [60]:
g25 = pd.read_csv('./data/census/gcpsa3/2016 Census GCP Statistical Area 3 for AUST/2016Census_G25_AUS_SA3.csv')
g25.head(5)

,SA3_CODE_2016,CF_no_children_F,CF_no_children_P,CF_ChU15_a_DSs_a_NdCh_F,CF_ChU15_a_DSs_a_NdCh_P,CF_ChU15_a_DSs_a_no_NdCh_F,CF_ChU15_a_DSs_a_no_NdCh_P,CF_ChU15_a_no_DSs_a_NdCh_F,CF_ChU15_a_no_DSs_a_NdCh_P,CF_ChU15_a_no_DSs_a_no_NdCh_F,...,OPF_no_ChU15_no_DSs_a_NdCh_F,OPF_no_ChU15_no_DSs_a_NdCh_P,OPF_no_ChU15_a_Total_F,OPF_no_ChU15_a_Total_P,OPF_Total_F,OPF_Total_P,Other_family_F,Other_family_P,Total_F,Total_P
0,10102,5577,10999,105,573,762,3495,185,844,3963,...,722,1562,994,2231,2072,5214,180,370,15092,44276
1,10103,2220,4334,29,151,180,803,34,157,1057,...,259,547,355,752,682,1712,52,103,4808,13190
2,10104,10377,20461,95,535,566,2675,204,922,2988,...,1026,2105,1387,2936,2929,7522,195,399,19216,50111
3,10105,3922,7788,70,397,345,1603,137,650,1861,...,554,1167,733,1565,1551,4039,129,263,9184,25950
4,10106,4093,8081,83,441,450,2102,126,575,2160,...,448,927,608,1317,1350,3588,118,240,9571,27619


In [61]:
persons_columns = list(filter(lambda e: e.endswith('_P'), g25.columns))
display(persons_columns)
display(len(persons_columns))

['CF_no_children_P',
 'CF_ChU15_a_DSs_a_NdCh_P',
 'CF_ChU15_a_DSs_a_no_NdCh_P',
 'CF_ChU15_a_no_DSs_a_NdCh_P',
 'CF_ChU15_a_no_DSs_a_no_NdCh_P',
 'CF_ChU15_a_Total_P',
 'CF_no_ChU15_a_DSs_a_NdCh_P',
 'CF_no_ChU15_a_DSs_a_no_NdCh_P',
 'CF_no_ChU15_a_no_DSs_a_NdCh_P',
 'CF_no_ChU15_a_Total_P',
 'CF_Total_P',
 'OPF_ChU15_a_DSs_a_NdCh_P',
 'OPF_ChU15_a_DSs_a_no_NdCh_P',
 'OPF_ChU15_a_no_DSs_a_NdCh_P',
 'OPF_ChU15_a_no_DSs_no_NdCh_P',
 'OPF_ChU15_a_Total_P',
 'OPF_no_ChU15_a_DSs_a_NdCh_P',
 'OPF_no_ChU15_a_DSs_no_NdCh_P',
 'OPF_no_ChU15_no_DSs_a_NdCh_P',
 'OPF_no_ChU15_a_Total_P',
 'OPF_Total_P',
 'Other_family_P',
 'Total_P']

23

In [62]:
a = ['CF_no_children_P', 'Couple without Dependants',
 'CF_ChU15_a_DSs_a_NdCh_P', 'Couple with Dependants',
 'CF_ChU15_a_DSs_a_no_NdCh_P', 'Couple with Dependants',
 'CF_ChU15_a_no_DSs_a_NdCh_P', 'Couple with Dependants',
 'CF_ChU15_a_no_DSs_a_no_NdCh_P', 'Couple with Dependants',
 #'CF_ChU15_a_Total_P',
 'CF_no_ChU15_a_DSs_a_NdCh_P', 'Couple with Dependants',
 'CF_no_ChU15_a_DSs_a_no_NdCh_P', 'Couple with Dependants',
 'CF_no_ChU15_a_no_DSs_a_NdCh_P', 'Couple without Dependants', 
 #'CF_no_ChU15_a_Total_P',
 #'CF_Total_P',
 'OPF_ChU15_a_DSs_a_NdCh_P', 'Single with Dependants',
 'OPF_ChU15_a_DSs_a_no_NdCh_P', 'Single with Dependants',
 'OPF_ChU15_a_no_DSs_a_NdCh_P', 'Single with Dependants',
 'OPF_ChU15_a_no_DSs_no_NdCh_P', 'Single with Dependants',
 #'OPF_ChU15_a_Total_P',
 'OPF_no_ChU15_a_DSs_a_NdCh_P', 'Single with Dependants',
 'OPF_no_ChU15_a_DSs_no_NdCh_P', 'Single with Dependants',
 'OPF_no_ChU15_no_DSs_a_NdCh_P', 'Single without Dependants',
 #'OPF_no_ChU15_a_Total_P',
 #'OPF_Total_P',
 #'Other_family_P',
#'Total_P'
    ]

In [63]:
b25_to_insolvencies = dict(zip(a[::2], a[1::2]))
from collections import defaultdict
insolvencies_to_b25 = defaultdict(list)
for a,b in (zip(a[1::2], a[::2])):
    insolvencies_to_b25[a].append(b)
#insolvencies_to_b25 = dict(zip(a[1::2], a[::2]))
display(b25_to_insolvencies)
display(insolvencies_to_b25)

{'CF_no_children_P': 'Couple without Dependants',
 'CF_ChU15_a_DSs_a_NdCh_P': 'Couple with Dependants',
 'CF_ChU15_a_DSs_a_no_NdCh_P': 'Couple with Dependants',
 'CF_ChU15_a_no_DSs_a_NdCh_P': 'Couple with Dependants',
 'CF_ChU15_a_no_DSs_a_no_NdCh_P': 'Couple with Dependants',
 'CF_no_ChU15_a_DSs_a_NdCh_P': 'Couple with Dependants',
 'CF_no_ChU15_a_DSs_a_no_NdCh_P': 'Couple with Dependants',
 'CF_no_ChU15_a_no_DSs_a_NdCh_P': 'Couple without Dependants',
 'OPF_ChU15_a_DSs_a_NdCh_P': 'Single with Dependants',
 'OPF_ChU15_a_DSs_a_no_NdCh_P': 'Single with Dependants',
 'OPF_ChU15_a_no_DSs_a_NdCh_P': 'Single with Dependants',
 'OPF_ChU15_a_no_DSs_no_NdCh_P': 'Single with Dependants',
 'OPF_no_ChU15_a_DSs_a_NdCh_P': 'Single with Dependants',
 'OPF_no_ChU15_a_DSs_no_NdCh_P': 'Single with Dependants',
 'OPF_no_ChU15_no_DSs_a_NdCh_P': 'Single without Dependants'}

defaultdict(list,
            {'Couple without Dependants': ['CF_no_children_P',
              'CF_no_ChU15_a_no_DSs_a_NdCh_P'],
             'Couple with Dependants': ['CF_ChU15_a_DSs_a_NdCh_P',
              'CF_ChU15_a_DSs_a_no_NdCh_P',
              'CF_ChU15_a_no_DSs_a_NdCh_P',
              'CF_ChU15_a_no_DSs_a_no_NdCh_P',
              'CF_no_ChU15_a_DSs_a_NdCh_P',
              'CF_no_ChU15_a_DSs_a_no_NdCh_P'],
             'Single with Dependants': ['OPF_ChU15_a_DSs_a_NdCh_P',
              'OPF_ChU15_a_DSs_a_no_NdCh_P',
              'OPF_ChU15_a_no_DSs_a_NdCh_P',
              'OPF_ChU15_a_no_DSs_no_NdCh_P',
              'OPF_no_ChU15_a_DSs_a_NdCh_P',
              'OPF_no_ChU15_a_DSs_no_NdCh_P'],
             'Single without Dependants': ['OPF_no_ChU15_no_DSs_a_NdCh_P']})

You can extract the data like this:

In [64]:
x = g25.loc[0]
#display(x)
x[['CF_no_children_P', 'CF_ChU15_a_DSs_a_NdCh_P']]
x[insolvencies_to_b25['Couple without Dependants']].sum()
# Apparently you can do this with .groupby

14915

### 3. Produce a new version of B25 whose columns are the family situations found in (1)

In [65]:
new_columns = ['SA3_CODE_2016', *family_situations]
def g25_to_insolvencies(row):
    data = [row['SA3_CODE_2016']]
    data.extend([row[insolvencies_to_b25[x]].sum() for x in family_situations])
    return data

yes_finally_why_was_this_so_hard = g25.apply(g25_to_insolvencies, axis=1, result_type='expand')
yes_finally_why_was_this_so_hard.columns = new_columns
display(yes_finally_why_was_this_so_hard.head(5))
yes_finally_why_was_this_so_hard.to_csv('./data/preprocess/b25_but_with_less_family_situation_granularity.csv', index=False)

,SA3_CODE_2016,Single with Dependants,Couple without Dependants,Couple with Dependants,Single without Dependants
0,10102,3658,14915,23770,1562
1,10103,1164,5292,6082,547
2,10104,5422,24122,18069,2105
3,10105,2863,10250,11383,1167
4,10106,2660,10354,13446,927


### Add sub-major group column to non-compliance data

In [ ]:
import numpy as np
non_compliance_data['Debtor Occupation Code (Major) (ANZSCO)'] = non_compliance_data['Debtor Occupation Code (ANZSCO)'].apply(lambda e: np.nan if np.isnan(e) else int(str(e)[0])).fillna(10.0)
non_compliance_data.to_csv('./data/preprocess/non-compliance-major-group.csv', index=False)